# Dataset Creation

## Table of Contents
* [Requirements](#requirements)
* [Data Collection](#data-collection)
    * [Weather Data](#weather-data)
    * [Day-Ahead Price Data](#day-ahead-price-data)
    * [Production Data](#production-data)
    * [Cross-Border Electricity Trading Data](#cross-border-electricity-trading-data)
* [Data Concatenation](#data-concatenation)
* [Data Dictionary](#data-dictionary)

----

## Requirements

In [1]:
import requests
from functools import reduce

import pandas as pd


pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', 100) 

/Users/benleidig/Downloads/ures_temp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


----

## Data Collection

### Weather Data

In [2]:
coordinates = [
    (52.5200, 13.4050),  # Berlin
    (53.5488, 9.9872),   # Hamburg
    (48.1351, 11.5820),  # Munich
    (50.9375, 6.9603)    # Cologne
]
weather_variables = [
    'precipitation',
    'cloud_cover',
    'sunshine',
    'temperature',
    'relative_humidity'
]

In [3]:
%%time

url = 'https://api.brightsky.dev/weather'
parameters = {
    'date':'2024-01-01T01:00:00Z',
    'last_date':'2025-01-01T00:00:00Z'
}
concat_list = []
for lat, lon in coordinates:
    parameters['lat'] = lat
    parameters['lon'] = lon
    data = requests.get(url, parameters).json()
    temp = pd.DataFrame(data['weather'])[['timestamp'] + weather_variables]
    concat_list.append(temp)
weather = pd.concat(concat_list)
weather['datetime'] = pd.to_datetime(weather['timestamp'], format='ISO8601', utc=True) + pd.Timedelta(hours=-1)
weather = weather\
    .drop(columns='timestamp')\
    .groupby('datetime', as_index=False)\
    .agg('mean')
print(weather.shape)
weather.head()

(8784, 6)
CPU times: user 472 ms, sys: 43.1 ms, total: 515 ms
Wall time: 33.1 s


,datetime,precipitation,cloud_cover,sunshine,temperature,relative_humidity
0,2024-01-01 00:00:00+00:00,0.00,96.75,0.0,6.250,77.75
1,2024-01-01 01:00:00+00:00,0.00,100.00,0.0,6.125,77.25
2,2024-01-01 02:00:00+00:00,0.25,81.00,0.0,5.575,81.50
3,2024-01-01 03:00:00+00:00,0.15,56.25,0.0,5.750,79.75
4,2024-01-01 04:00:00+00:00,0.00,74.75,0.0,6.150,77.25


### Day-Ahead Price Data

In [4]:
%%time

url = 'https://api.energy-charts.info/price'
parameters = {
    'start':'2024-01-01T00:00:00Z',
    'end':'2024-12-31T23:00:00Z'
}
data = requests.get(url, params=parameters).json()
data.pop('license_info'); data.pop('unit'); data.pop('deprecated')
price = pd.DataFrame(data)
price['datetime'] = pd.to_datetime(price['unix_seconds'], unit='s', utc=True)
price = price.drop(columns='unix_seconds')
print(price.shape)
price.head()

(8784, 2)
CPU times: user 29.8 ms, sys: 3.9 ms, total: 33.7 ms
Wall time: 2.46 s


,price,datetime
0,0.01,2024-01-01 00:00:00+00:00
1,0.00,2024-01-01 01:00:00+00:00
2,-0.01,2024-01-01 02:00:00+00:00
3,-0.03,2024-01-01 03:00:00+00:00
4,-0.02,2024-01-01 04:00:00+00:00


### Production Data

In [3]:
%%time

url = 'https://api.energy-charts.info/public_power'
parameters = {
    'start':'2024-01-01T01:00:00Z',
    'end':'2025-01-01T00:00:00Z'
}
data = requests.get(url, params=parameters).json()
public_power = pd.DataFrame()
public_power['datetime'] = pd.to_datetime(data['unix_seconds'], unit='s', utc=True) + pd.Timedelta(hours=-1)
for production_type_data in data['production_types']:
    col = production_type_data['name'].lower().replace(' ', '_').replace('-', '_')
    public_power[col] = production_type_data['data']
public_power = public_power[
    public_power['datetime'].dt.minute == 0
].drop(columns=['hydro_pumped_storage_consumption', 'cross_border_electricity_trading'])
print(public_power.shape)
public_power.head()

(8784, 20)
CPU times: user 270 ms, sys: 33.6 ms, total: 304 ms
Wall time: 3.97 s


,datetime,hydro_run_of_river,biomass,fossil_brown_coal_/_lignite,fossil_hard_coal,fossil_oil,fossil_coal_derived_gas,fossil_gas,geothermal,hydro_water_reservoir,hydro_pumped_storage,others,waste,wind_offshore,wind_onshore,solar,load,residual_load,renewable_share_of_load,renewable_share_of_generation
0,2024-01-01 00:00:00+00:00,2370.4,4041.2,3332.3,1548.2,398.1,412.0,2398.9,20.2,223.0,451.6,152.4,798.8,5366.0,29023.9,0.0,38408.0,4018.1,107.9,82.0
4,2024-01-01 01:00:00+00:00,2371.4,4011.2,3354.8,1539.4,398.3,423.0,2425.0,20.4,208.4,379.0,152.3,797.2,4611.7,28899.0,0.0,37307.9,3797.3,108.6,81.7
8,2024-01-01 02:00:00+00:00,2344.2,4013.5,3353.9,1541.9,398.1,423.9,2426.8,20.3,203.6,164.8,152.3,804.3,4584.1,28411.8,0.0,36984.6,3988.6,108.0,81.8
12,2024-01-01 03:00:00+00:00,2347.4,4044.1,3359.3,1542.8,398.4,398.6,2429.4,19.5,203.1,159.8,152.4,798.5,4566.2,28552.9,2.5,37366.9,4245.3,107.4,81.9
16,2024-01-01 04:00:00+00:00,2333.7,4104.8,3384.7,1550.7,397.8,401.0,2445.2,19.2,135.8,144.3,152.4,799.8,4496.3,28786.7,2.7,36592.6,3306.9,110.0,81.9


### Cross-Border Electricity Trading Data

In [6]:
%%time

url = 'https://api.energy-charts.info/cbet'
parameters = {
    'start':'2024-01-01T01:00:00Z',
    'end':'2025-01-01T00:00:00Z'
}
data = requests.get(url, parameters).json()
cbet = pd.DataFrame()
cbet['datetime'] = pd.to_datetime(data['unix_seconds'], unit='s', utc=True) + pd.Timedelta(hours=-1)
for country_data in data['countries']:
    col = country_data['name'].lower().replace(' ', '_').replace('-', '_') + '_cbet'
    cbet[col] = country_data['data']
cbet = cbet[
    cbet['datetime'].dt.minute == 0
]
print(cbet.shape)
cbet.head()

(8784, 13)
CPU times: user 182 ms, sys: 12.8 ms, total: 195 ms
Wall time: 23.5 s


,datetime,austria_cbet,belgium_cbet,czech_republic_cbet,denmark_cbet,france_cbet,luxembourg_cbet,netherlands_cbet,norway_cbet,poland_cbet,sweden_cbet,switzerland_cbet,sum_cbet
0,2024-01-01 00:00:00+00:00,-1.608,-1.000,-0.826,-3.500,-0.732,-0.186,-0.309,-1.4,-0.773,-0.519,-0.796,-11.650
4,2024-01-01 01:00:00+00:00,-1.022,-1.000,-0.894,-3.458,-1.214,-0.182,-0.157,-1.4,-0.847,-0.519,-0.800,-11.493
8,2024-01-01 02:00:00+00:00,-0.817,-0.663,-0.607,-3.398,-1.900,-0.184,0.582,-1.4,-0.915,-0.519,-0.794,-10.616
12,2024-01-01 03:00:00+00:00,-0.566,-0.643,-0.737,-3.000,-2.900,-0.194,0.230,-1.4,-0.896,-0.519,-0.800,-11.426
16,2024-01-01 04:00:00+00:00,-0.811,-0.998,-0.654,-3.090,-0.548,-0.194,-0.857,-1.4,-0.772,-0.519,-0.799,-10.642


----

## Data Concatenation

In [7]:
df = reduce(
    lambda l, r : l.merge(r, on='datetime', how='outer'),
    [weather, price, public_power, cbet]
).sort_values(by='datetime', ascending=True)
df.to_csv('dataset.csv', index=False)
print(df.shape)
df.head()

(8784, 38)


,datetime,precipitation,cloud_cover,sunshine,temperature,relative_humidity,price,hydro_run_of_river,biomass,fossil_brown_coal_/_lignite,fossil_hard_coal,fossil_oil,fossil_coal_derived_gas,fossil_gas,geothermal,hydro_water_reservoir,hydro_pumped_storage,others,waste,wind_offshore,wind_onshore,solar,load,residual_load,renewable_share_of_load,renewable_share_of_generation,austria_cbet,belgium_cbet,czech_republic_cbet,denmark_cbet,france_cbet,luxembourg_cbet,netherlands_cbet,norway_cbet,poland_cbet,sweden_cbet,switzerland_cbet,sum_cbet
0,2024-01-01 00:00:00+00:00,0.00,96.75,0.0,6.250,77.75,0.01,2370.4,4041.2,3332.3,1548.2,398.1,412.0,2398.9,20.2,223.0,451.6,152.4,798.8,5366.0,29023.9,0.0,38408.0,4018.1,107.9,82.0,-1.608,-1.000,-0.826,-3.500,-0.732,-0.186,-0.309,-1.4,-0.773,-0.519,-0.796,-11.650
1,2024-01-01 01:00:00+00:00,0.00,100.00,0.0,6.125,77.25,0.00,2371.4,4011.2,3354.8,1539.4,398.3,423.0,2425.0,20.4,208.4,379.0,152.3,797.2,4611.7,28899.0,0.0,37307.9,3797.3,108.6,81.7,-1.022,-1.000,-0.894,-3.458,-1.214,-0.182,-0.157,-1.4,-0.847,-0.519,-0.800,-11.493
2,2024-01-01 02:00:00+00:00,0.25,81.00,0.0,5.575,81.50,-0.01,2344.2,4013.5,3353.9,1541.9,398.1,423.9,2426.8,20.3,203.6,164.8,152.3,804.3,4584.1,28411.8,0.0,36984.6,3988.6,108.0,81.8,-0.817,-0.663,-0.607,-3.398,-1.900,-0.184,0.582,-1.4,-0.915,-0.519,-0.794,-10.616
3,2024-01-01 03:00:00+00:00,0.15,56.25,0.0,5.750,79.75,-0.03,2347.4,4044.1,3359.3,1542.8,398.4,398.6,2429.4,19.5,203.1,159.8,152.4,798.5,4566.2,28552.9,2.5,37366.9,4245.3,107.4,81.9,-0.566,-0.643,-0.737,-3.000,-2.900,-0.194,0.230,-1.4,-0.896,-0.519,-0.800,-11.426
4,2024-01-01 04:00:00+00:00,0.00,74.75,0.0,6.150,77.25,-0.02,2333.7,4104.8,3384.7,1550.7,397.8,401.0,2445.2,19.2,135.8,144.3,152.4,799.8,4496.3,28786.7,2.7,36592.6,3306.9,110.0,81.9,-0.811,-0.998,-0.654,-3.090,-0.548,-0.194,-0.857,-1.4,-0.772,-0.519,-0.799,-10.642


In [8]:
df.isna().sum()

datetime                         0
precipitation                    0
cloud_cover                      0
sunshine                         0
temperature                      0
relative_humidity                0
price                            0
hydro_run_of_river               0
biomass                          0
fossil_brown_coal_/_lignite      0
fossil_hard_coal                 0
fossil_oil                       0
fossil_coal_derived_gas          0
fossil_gas                       0
geothermal                       0
hydro_water_reservoir            0
hydro_pumped_storage             0
others                           0
waste                            0
wind_offshore                    0
wind_onshore                     0
solar                            0
load                             0
residual_load                    0
renewable_share_of_load          0
renewable_share_of_generation    0
austria_cbet                     0
belgium_cbet                     0
czech_republic_cbet 

----

## Data Dictionary

In [ ]:
descriptions_units = {
    'datetime':                         ('ISO 8601-formatted timestamp of this record in UTC.', 'YYYY-MM-DDThh:mm:ss'),
    'precipitation':                    ('Total precipitation during the following hour.', 'mm'),
    'cloud_cover':                      ('Total cloud cover an hour after the timestamp.', '%'),
    'sunshine':                         ('Sunshine duration during the following hour.', 'min'),
    'temperature':                      ('Air temperature an hour after the timestamp, 2 m above the ground.', 'ºC'),
    'relative_humidity':                ('Relative humidity an hour after the timestamp.', '%'),
    'price':                            ('The day-ahead spot market price for the DE-LU bidding zone. Takes the last 15-minute interval value and assigns to the start of the hour interval.', 'EUR/MWh'),
    'hydro_run_of_river':               ('Electricity generation from hydro run-of-river recorded at the end of the following hour.', 'MW'),
    'biomass':                          ('Electricity generation from biomass recorded at the end of the following hour.', 'MW'),
    'fossil_brown_coal_/_lignite':      ('Electricity generation from fossil brown coal / lignite recorded at the end of the following hour.', 'MW'),
    'fossil_hard_coal':                 ('Electricity generation from fossil hard coal recorded at the end of the following hour.', 'MW'),
    'fossil_oil':                       ('Electricity generation from fossil oil recorded at the end of the following hour.', 'MW'),
    'fossil_coal_derived_gas':          ('Electricity generation from fossil coal-derived gas recorded at the end of the following hour.', 'MW'),
    'fossil_gas':                       ('Electricity generation from fossil gas recorded at the end of the following hour.', 'MW'),
    'geothermal':                       ('Electricity generation from geothermal recorded at the end of the following hour.', 'MW'),
    'hydro_water_reservoir':            ('Electricity generation from hydro water reservoir recorded at the end of the following hour.', 'MW'),
    'hydro_pumped_storage':             ('Electricity generation from pumped storage recorded at the end of the following hour.', 'MW'),
    'others':                           ('Electricity generation from other sources recorded at the end of the following hour.', 'MW'),
    'waste':                            ('Electricity generation from waste recorded at the end of the following hour.', 'MW'),
    'wind_offshore':                    ('Electricity generation from wind offshore recorded at the end of the following hour.', 'MW'),
    'wind_onshore':                     ('Electricity generation from wind onshore recorded at the end of the following hour.', 'MW'),
    'solar':                            ('Electricity generation from solar recorded at the end of the following hour.', 'MW'),
    'load':                             ('Instantaneous grid load recorded at the end of the following hour.', 'MW'),
    'residual_load':                    ('Instantaneous grid load not covered by renewable generation sources recorded at the end of the following hour.', 'MW'),
    'renewable_share_of_load':          ('Instantaneous grid load covered by renewable generation sources recorded at the end of the following hour.', 'MW'),
    'renewable_share_of_generation':    ('Total electricity generation from renewable generation sources recorded at the end of the following hour.', 'MW'),
    'austria_cbet':                     ('The cross-border electricity trading with Austria recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'belgium_cbet':                     ('The cross-border electricity trading with Belgium recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'czech_republic_cbet':              ('The cross-border electricity trading with Czech Republic recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'denmark_cbet':                     ('The cross-border electricity trading with Denmark recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'france_cbet':                      ('The cross-border electricity trading with France recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'luxembourg_cbet':                  ('The cross-border electricity trading with Luxembourg recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'netherlands_cbet':                 ('The cross-border electricity trading with Netherlands recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'norway_cbet':                      ('The cross-border electricity trading with Norway recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'poland_cbet':                      ('The cross-border electricity trading with Poland recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'sweden_cbet':                      ('The cross-border electricity trading with Sweden recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'switzerland_cbet':                 ('The cross-border electricity trading with Switzerland recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW'),
    'sum_cbet':                         ('The cross-border electricity trading with all other countries recorded at the end of the following hour. Positive values indicate an import of electricity, whereas negative values show electricity exports.', 'GW')
}

data_dict = {}
for var in df.columns:
    data_dict[var] = [df[var].dtype, descriptions_units[var][1], descriptions_units[var][0]]

data_dict_df = pd.DataFrame.from_dict(
    data_dict,
    orient='index',
    columns=['dtype', 'units', 'description']
)
data_dict_df.to_csv('data_dictionary.csv')
print(data_dict_df.shape)
data_dict_df.head(100)

(38, 3)


,dtype,units,description
datetime,"datetime64[ns, UTC]",YYYY-MM-DDThh:mm:ss,ISO 8601-formatted timestamp of this record in...
precipitation,float64,mm,Total precipitation during the following hour.
cloud_cover,float64,%,Total cloud cover an hour after the timestamp.
sunshine,float64,min,Sunshine duration during the following hour.
temperature,float64,ºC,"Air temperature an hour after the timestamp, 2..."
relative_humidity,float64,%,Relative humidity an hour after the timestamp.
price,float64,EUR/MWh,The day-ahead spot market price for the DE-LU ...
hydro_run_of_river,float64,MW,Electricity generation from hydro run-of-river...
biomass,float64,MW,Electricity generation from biomass recorded a...
fossil_brown_coal_/_lignite,float64,MW,Electricity generation from fossil brown coal ...
